# Compresión de Imágenes
Comprimir una imagen es reducir los datos redundantes e irrelevantes de la imagen con la menor pérdida posible, para permitir su almacenamiento o transmisión de forma eficiente.

![title](media/compresion1.svg)

En el curso, se explicará un método para comprimir imágenes llamado JPEG, que utiliza la transformada discreta de coseno de 2 dimensiones (DCT-2D).

## JPEG
- Joint Photographic Experts Group
- Es el nombre de un comité de expertos que creó un estándar de compresión y codificación de imágenes.
- Al inicio JPEG fue considerado un método de compresión, pero después fue considerado un formato de archivo. Los archivos de este tipo suelen almacenarse con la extensión .jpg
- El formato JPEG utiliza un algoritmo de compresión de pérdida para reducir el tamaño de los archivos de imágenes. Esto significa que al descomprimir o visualizar la imagen no se obtiene la imagen original, sino una aproximación.
- Este algoritmo se basa en el fenómeno del ojo humano de detectar más fácilmente los valores de baja frecuencia.
- Este método usa dos conceptos primordiales: DCT-2D y cuantización.

## DCT-2D
Sea $A \in {\rm I\!R}^{m \times m}$. Entonces la DCT-2D de $A$ se define como $C \in {\rm I\!R}^{m \times m}$ tal que

\begin{equation}
C_{(i,j)}=\dfrac{1}{\sqrt{2m}}\cdot C_{(i)}\cdot C_{(j)} \cdot \sum_{x=0}^{m-1} \sum_{y=0}^{m-1} A(x,y)\cdot \cos{\left(\dfrac{(2x+1)i \pi}{2m}\right)} \cos{\left( \dfrac{(2y+1)j \pi}{2m} \right)}
\end{equation}

donde 

\begin{equation}
C(k)=\left\{\begin{matrix}
\dfrac{1}{\sqrt{2}} & \text{si} & k=0\\ 
1 & \text{si} & k>0 
\end{matrix}\right.
\end{equation}

## Cuantización
Es una técnica de compresión con pérdida que consiste en comprimir un rango de valores a un grupo reducido de valores. En este caso, se necesita una matriz de cuantificación $Q$. Entonces para cuantificar una matriz $A$, se realiza la siguiente operación

\begin{equation}
D_{i,j} = \text{redondear}\left( \dfrac{A_{i,j}}{Q_{i,j}} \right)
\end{equation}

## Algoritmo JPEG
### Compresión
- Paso 0. Cargar matriz en formato double, donde las entradas toman valores del conjunto \{0,1,2,...,255\}. Sea esta matriz $A \in {\rm I\!R}^{m \times m}$
- Paso 1. Restar a cada una de las entradas el valor 128 (ya que la DCT-2D está diseñada para trabajar con valores en ]-128, 127]). Sea esta matriz $M \in {\rm I\!R}^{m \times m}$, donde

\begin{equation}
M_{i,j}=A_{i,j} - 128
\end{equation}

- Paso 2. Calcular la DCT-2D de M: Sea esta matriz $D \in {\rm I\!R}^{m \times m}$, donde 

\begin{equation}
D = dct2(M)
\end{equation}

- Paso 3. Utilizando una matriz de cuantificación $Q \in {\rm I\!R}^{m \times m}$, se obtiene la matriz cuantificada $C \in {\rm I\!R}^{m \times m}$, donde

\begin{equation}
C_{i,j} = \dfrac{D_{i,j}}{Q_{i,j}}
\end{equation}

- Paso 4. Codificar la matriz C en un vector $x \in {\rm I\!R}^{p}$, $p << m^2$, utilizando el método Zig-Zag.

### Matriz de cuantificación Q
La matriz de cuantificación es una matriz que permite comprimir valores. Realizando estudios con la visión humana, investigadores obtuvieron que la siguiente matriz permite realizar una compresión del 50% de una imagen 8x8.

In [1]:
% Matriz de cuantificacion
Q50=[16 11 10 16 24 40 51 61;
     12 12 14 19 26 58 60 55;
     14 13 16 24 40 57 59 56;
     14 17 22 29 51 87 80 62;
     18 22 37 56 68 109 103 77;
     24 35 55 64 81 104 113 92;
     49 64 78 87 103 121 120 101;
     72 92 95 98 112 100 103 99];

Para variar la calidad de la cuantificación, se utiliza la siguiente fórmula

\begin{equation}
Q_n = \left\{\begin{matrix}
\text{redondear}\left[ \dfrac{(100-n)}{50}\cdot Q_{50} \right] & \text{si} & n>50\\ 
\text{redondear}\left[ \dfrac{50}{n}\cdot Q_{50} \right] & \text{si} & n<50
\end{matrix}\right.
\end{equation}

donde $n$ representa la calidad de compresión, $n=0,1,2,\ldots,100$

### Método de Zig-Zag
Este método se utiliza para almacenar en un vector los valores de la matriz $A$, cuando los valores diferentes de $0$ se encuentran agrupados en la parte superior izquierda de esta

![title](media/compresion2.svg)

**Nota.** En general, una matriz $A \in {\rm I\!R}^{m \times m}$ tiene $2m-1$ diagonales.

In [2]:
% Ejemplo de algoritmo JPEG para matrices de 8x8
clc; clear; close all;
pkg load signal;

% Compresion de la imagen

% Paso 0: Imaagen en formato double con valores en {0,1,...,255}
A=[154 123 123 123 123 123 123 136;
   192 180 136 154 154 154 136 110;
   254 198 154 154 180 154 123 123;
   239 180 136 180 180 166 123 123;
   180 154 136 167 166 149 136 136;
   128 136 123 136 154 180 198 154;
   123 105 110 149 136 136 180 166;
   110 136 123 123 123 136 154 136]

% Paso 1: Restar 128 en cada una de las entradas de la matriz A
M = A-128;

% Paso 2: Calcular la DCT-2D de M
D = dct2(M);

% Paso 3: Calcular la matriz cuantificada

Q50=[16 11 10 16 24 40 51 61;
     12 12 14 19 26 58 60 55;
     14 13 16 24 40 57 59 56;
     14 17 22 29 51 87 80 62;
     18 22 37 56 68 109 103 77;
     24 35 55 64 81 104 113 92;
     49 64 78 87 103 121 120 101;
     72 92 95 98 112 100 103 99]; % Matriz de cuantificacion
     
Q = Q50;
C = round(D./Q);

% Paso 4: Codificacion en vector x
m = 8;
x = [C(1,1)]; s = sum(sum(abs(C)));

for i = 2:2*m-1 % Recorrer las diagonales de C
  for j = 1:i % Recorrer los valores de cada una de las diagonales
    if mod(i,2) == 0 % Es una diagonal par?
      x = [x C(j,i-j+1)];
    else % Es una diagonal impar
      x = [x C(i-j+1,j)];
    end  
  end
  if sum(abs(x)) == s;
    break;
  end  
end  




% Proceso de reconstruccion

%% Obtener la matriz C, a partir del vector x

% Paso 1: Decodificar el vector x

% Paso 2: Multiplicar puntualmente la matriz C con la matriz Q
M = Q.*C;

% Paso 3: Calcular la inversa de la DCT-2D y redondear
N = round(idct2(M));

% Paso 4: Sumar 128 a cada una de las entradas
Ar = N+128

A =

   154   123   123   123   123   123   123   136
   192   180   136   154   154   154   136   110
   254   198   154   154   180   154   123   123
   239   180   136   180   180   166   123   123
   180   154   136   167   166   149   136   136
   128   136   123   136   154   180   198   154
   123   105   110   149   136   136   180   166
   110   136   123   123   123   136   154   136

Ar =

   149   134   119   116   121   126   127   128
   204   168   140   144   155   150   135   125
   253   195   155   166   183   165   131   111
   245   185   148   166   184   160   124   107
   188   149   132   155   172   159   141   136
   132   123   125   143   160   166   168   171
   109   119   126   128   139   158   168   166
   111   127   127   114   118   141   147   135

